In [1]:
import json
import numpy as np
import os
from tqdm import tqdm

In [2]:
PROMPT_PATH = "../prompts"

In [3]:
datasets = {
    "dtd": [2, 4, 8, 16, 32, 46, 60], 
    "fgvc_aircraft": [2, 4, 8, 14, 20, 30], 
    "sun397": [2, 4, 8, 16, 23, 30], 
    "flowers": [2, 4, 8, 14, 20, 30],
}

n_seeds = 10

In [7]:
# for dataset in ["flowers", "fgvc_aircraft", "dtd"]:
#     with open(f"../prompts/{dataset}/{dataset}_llama3_prompts_full.json", "r") as f:
#         prompt_dict = json.load(f)
#     with open(f"../prompts/{dataset}/{dataset}_llama3_prompts_full.json", "w") as f:
#         json.dump(prompt_dict, f, indent=4)

In [8]:
for dataset in ["flowers", "fgvc_aircraft", "dtd"]:
    with open(f"../prompts/{dataset}/{dataset}_llama3_prompts_full.json", "r") as f:
        prompt_dict = json.load(f)[dataset]

    for sample_size in datasets[dataset]:
        for seed in tqdm(range(n_seeds)):
            output = {classname.replace("(", "").replace(")", ""): [] for classname in prompt_dict}
            for c, classname in enumerate(prompt_dict):
                idx = np.random.RandomState(seed + c).choice(datasets[dataset][-1], sample_size, replace=True)
                for i in idx:
                    output[classname.replace("(", "").replace(")", "")].append(prompt_dict[classname][i])
            fp = os.path.join(PROMPT_PATH, f"{dataset}/sample_size_{sample_size:02d}_seed_{seed}.json") 
            with open(fp, 'w') as file:
                json.dump({dataset: output}, file, indent=4)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 98.53it/s]
